<a href="https://colab.research.google.com/github/chz23/HTR/blob/master/01_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check that it is running on GPU.

!nvidia-smi

Wed Jun 14 04:01:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install the required libraries.
# There will be a warning to restart runtime. Don't miss it out!

!pip install roboflow
!pip install ultralytics==8.0.20

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import pandas as pd
import numpy as np

from getpass import getpass

import glob
from roboflow import Roboflow
import ultralytics
from ultralytics import YOLO
from IPython.display import display, Image

from google.colab import files

In [4]:
# Create the relevant folders to store the data that will be downloaded below.

os.makedirs('./datasets')
os.chdir('./datasets')
os.getcwd()

'/content/datasets'

In [5]:
# This step allows for extracting cleaned, labelled & augmented data hosted on Roboflow.
# Requires an api key.

secret = getpass('Enter the api key: ')
rf = Roboflow(api_key=secret)
project = rf.workspace("dsi-hac9f").project("object-detection-for-htr")
dataset = project.version(5).download("yolov8")

Enter the api key: ··········
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Object-Detection-for-HTR-5 in yolov8:: 100%|██████████| 1192/1192 [00:00<00:00, 2135.31it/s]


In [6]:
os.chdir('..')
os.getcwd()

'/content'

In [ ]:
# Train the YOLOv8 model on the train set.

!yolo task=detect \
mode=train \
model=yolov8s.pt \
data={dataset.location}/data.yaml \
epochs=100 \
save=True \
save_crop=True \
hide_labels=True \
plots=True \
# conf=0.001 \ # default 0.001
iou=0.85 \ # default 0.7
save_conf=True \ # default False
hide_conf=True \ #default False
line_thickness=1 \ #default 3
box=7.5 \ # default 7.5
cls=0.5 \ #default 0.5
dfl=1.5 \ #default 1.5

100% 21.5M/21.5M [00:00<00:00, 166MB/s]

Ultralytics YOLOv8.0.20 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/content/datasets/Object-Detection-for-HTR-5/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=True, hide_labels=True, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=

In [ ]:
# Validate the trained model on the validation set.

!yolo task=detect \
mode=val \
model=./runs/detect/train/weights/best.pt \
data={dataset.location}/data.yaml \
save_json=True

2023-05-25 12:57:26.705808: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 12:57:27.632910: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/datasets/Object-Detection-for-HTR-5/valid/labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100% 49/49 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 4/4 [00:04<00:00,  1.10s/it]
                   all         49        503      0.989      0.984      0.993      0.853
Speed: 9.5ms pre-process, 27.4ms infe

In [ ]:
# Generate predictions on the test set.

!yolo task=detect \
mode=predict \
model=./runs/detect/train/weights/best.pt \
conf=0.5 \
source={dataset.location}/test/images \
save=True \
save_txt=True \
save_conf=True \
save_crop=True \
hide_labels=True \
hide_conf=True \ #default False?
line_thickness=1 \ #default 3?

2023-05-25 13:01:07.264022: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 13:01:08.516367: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
/usr/local/lib/python3.10/dist-packages/ultralytics/yolo/utils/plotting.py:160: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xyxy = torch.tensor(xyxy).view(-1, 4)
image 1/25 /content/datasets/Object-Detection-for-HTR-5/test/images/008_png.rf.9

In [ ]:
# Export the trained model for future use.

!yolo export model=./runs/detect/train/weights/best.pt

WARNING ⚠️ 'format' is missing. Using default 'format=torchscript'.
2023-05-25 13:03:17.202705: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 13:03:18.114946: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Ultralytics YOLOv8.0.20 🚀 Python-3.10.11 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from runs/detect/train/weights/best.pt with output shape (1, 5, 8400) (21.5 MB)

TorchScript: starting export with torch 2.0.1+cu118...
TorchScript: export success ✅ 1.2s, saved as runs/detect/train/weights/best.torchscript (42.8 MB)

Export complete (5.4s)
Results saved to /content/runs/detect/train/weights

In [ ]:
# Zip all the files produced from the training, validation & prediction processes.

!zip -r ./runs.zip ./runs

  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/val_batch1_pred.jpg (deflated 14%)
  adding: runs/detect/train/args.yaml (deflated 50%)
  adding: runs/detect/train/weights/ (stored 0%)
  adding: runs/detect/train/weights/best.pt (deflated 8%)
  adding: runs/detect/train/weights/last.pt (deflated 8%)
  adding: runs/detect/train/weights/best.torchscript (deflated 16%)
  adding: runs/detect/train/F1_curve.png (deflated 16%)
  adding: runs/detect/train/events.out.tfevents.1685017192.bad3bea5253c.1108.0 (deflated 72%)
  adding: runs/detect/train/train_batch1.jpg (deflated 12%)
  adding: runs/detect/train/results.csv (deflated 86%)
  adding: runs/detect/train/val_batch0_pred.jpg (deflated 13%)
  adding: runs/detect/train/train_batch0.jpg (deflated 11%)
  adding: runs/detect/train/R_curve.png (deflated 18%)
  adding: runs/detect/train/train_batch2971.jpg (deflated 14%)
  adding: runs/detect/train/PR_curve.png 

In [ ]:
# Download the zip file.

files.download("./runs.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>